In [ ]:
from ast import parse
import bs4
from langchain_community.document_loaders import WebBaseLoader
import torch
import tqdm
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

from langchain.load import dumps, loads
from langchain.schema.output_parser import StrOutputParser

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import Ollama
import os
from operator import itemgetter


os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['USER_AGENT'] = ''


loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_ = (
                "post-content", "post-title", "post-header"
            )
        )
    ),
)

blog_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50
)

splits = text_splitter.split_documents(blog_docs)


modelPath = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

### ___Query LLM___

In [ ]:
#model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model_id = "microsoft/Phi-3-mini-4k-instruct"
Query_LLM = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    model_kwargs={ "temperature": 0.0,
                   "do_sample": True },
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


### ___Answer LLM___

In [ ]:
Answer_LLM = Ollama(
    model="qwen2.5:7b-instruct",
    temperature=0.2
)

#print(Answer_LLM("Explain transformers in simple terms"))

# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
# model_id = "microsoft/Phi-3-mini-4k-instruct"

# Answer_LLM = HuggingFacePipeline.from_model_id(
#     model_id=model_id,
#     task="text-generation",
#     model_kwargs={ "temperature": 0.0,
#                    "do_sample": True },
# )

# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(model_id)

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=256,
#     temperature=0.0,
# )

# Answer_LLM = HuggingFacePipeline(pipeline=pipe)

C:\Users\sarma\AppData\Local\Temp\ipykernel_14912\3409403279.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  Answer_LLM = Ollama(
C:\Users\sarma\AppData\Local\Temp\ipykernel_14912\3409403279.py:6: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(Answer_LLM("Explain transformers in simple terms"))


Sure! Transformers are a type of neural network architecture used primarily for processing sequences of data, like text or speech. They were introduced in 2017 and have since become very popular because they can handle tasks more efficiently than previous models.

Here’s a simple explanation:

Imagine you're reading a book. To understand the story, you need to remember what happened earlier while focusing on the current page. Traditional methods would require you to reread parts of the book multiple times or keep notes, which is time-consuming.

Transformers work differently. They use something called "self-attention," which allows them to focus on different words in a sentence based on their importance and context. It's like being able to instantly recall any part of the story without needing to reread it.

Here’s how it works:
1. **Input**: You give the transformer a sequence, like a sentence.
2. **Self-Attention**: The model looks at each word and decides which other words in the se

### __RAG-Fusion__

In [ ]:
template = """
You are an AI language model assistant. Generate multiple search queries related 
Original question: {question}
output (4 queries):
"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion
    | Query_LLM
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

def reciprocal_rank_fusion(results: list[list], k = 60):
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)
        
        reranked_results = [
            (load(doc), score)
            for doc, score in sorted(fused_scores.items(), key = lambda x: x[1], reverse=True)
        ]

        return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke( { "question": question } )
len(docs)

In [ ]:
template = """
Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    { "context" :  retrieval_chain_rag_fusion,
      "question": itemgetter("question")
     }
    | prompt
    | Answer_LLM
    | StrOutputParser()
)

final_rag_chain.invoke( { "question": question } )
